In [2]:
from pathlib import Path
import pandas as pd
import numpy as np

csv_path = Path.cwd().parent / "data" / "driving_log.csv"

print("CWD:", Path.cwd())
print("CSV:", csv_path)
print("EXISTS:", csv_path.exists())

if not csv_path.exists():
    raise FileNotFoundError(f"CSV not found: {csv_path}")

df = pd.read_csv(csv_path)

# 2025-08-09
# 로그 스키마가 바뀌면 뒤에서 전부 깨지므로, 초반에 필수 컬럼부터 강제 체크.
required_cols = {"time_sec", "speed_kmh", "accel_mps2"}
missing = required_cols - set(df.columns)
if missing:
    raise ValueError(f"Missing columns: {missing}")

# 2025-08-10
# 임계값은 이전 프로젝트들과 비교 가능하도록 동일한 값 유지.
# 점수 비교가 목적이라 여기서는 파라미터화하지 않음.
HARD_BRAKE_THRESHOLD = -3.0   # m/s^2
HARD_ACCEL_THRESHOLD = 3.0    # m/s^2

# 2025-08-12
# 이벤트 탐지 자체는 단순하지만,
# 실제로는 연속 구간을 하나의 이벤트로 묶는 게 더 까다롭다.
# 이 버전에서는 샘플 단위 카운트로 단순화.
hard_brake_df = df[df["accel_mps2"] < HARD_BRAKE_THRESHOLD]
hard_accel_df = df[df["accel_mps2"] > HARD_ACCEL_THRESHOLD]

hard_brake_count = len(hard_brake_df)
hard_accel_count = len(hard_accel_df)
total_hard_events = hard_brake_count + hard_accel_count

# 2025-08-14
# 주행 시간/거리 계산은 평균 속도로 근사.
# 정확도보다는 파이프라인 구조 확인이 목적.
total_time_sec = df["time_sec"].max() - df["time_sec"].min()
avg_speed_kmh = df["speed_kmh"].mean()
distance_km = avg_speed_kmh * (total_time_sec / 3600) if total_time_sec > 0 else 0

# 2025-08-16
# 짧은 로그에서 events/min, events/km 값이 튀는 문제가 가장 신경 쓰였다.
# 최소 주행 시간(60초) 조건을 만족하지 않으면 None 처리.
events_per_min = (total_hard_events / (total_time_sec / 60)) if total_time_sec >= 60 else None
events_per_km = (total_hard_events / distance_km) if distance_km > 0 else None

hard_brake_ratio = (hard_brake_count / total_hard_events) if total_hard_events > 0 else 0
speed_variability = df["speed_kmh"].std()

# 2025-08-18
# normalize는 단순하지만, 데이터 부족(None) 처리 방식이 결과를 크게 좌우한다.
# 여기서는 보수적으로 0점 처리.
def normalize(value, max_value):
    if value is None or max_value <= 0:
        return 0
    return min(float(value) / float(max_value), 1.0)

# 2025-08-20
# 공격성 점수는 포트폴리오용 가중치 조합.
# 점수 자체보다 "지표 결합 방식"과 안정성 처리를 보여주는 게 목적.
score = 0
score += normalize(events_per_km, 10) * 30
score += normalize(events_per_min, 5) * 30
score += hard_brake_ratio * 20
score += normalize(speed_variability, 20) * 20
aggressiveness_score = round(score, 1)

# 2025-08-22
# 점수 → 등급 매핑은 단순 기준.
# 실제 시스템에서는 캘리브레이션이 필요하지만,
# 포트폴리오에서는 의사결정 흐름 표현이 목적.
def classify_score(score):
    if score < 30:
        return "Calm Driver"
    elif score < 60:
        return "Normal Driver"
    else:
        return "Aggressive Driver"

driver_grade = classify_score(aggressiveness_score)

# 2025-08-24
# 결과 출력은 최종 점수뿐 아니라,
# 중간 지표까지 같이 보여주도록 구성.
print("Aggressive Driving Score Result")
print("=" * 45)
print(f"Data Path               : {csv_path}")
print(f"Total Time (sec)         : {total_time_sec:.1f}")
print(f"Distance (km)            : {distance_km:.2f}")
print("-" * 45)
print(f"Hard Accel Events        : {hard_accel_count}")
print(f"Hard Brake Events        : {hard_brake_count}")
print(f"Total Hard Events        : {total_hard_events}")
print("-" * 45)
print(f"Events per Minute        : {events_per_min if events_per_min is not None else 'Insufficient Data'}")
print(f"Events per km            : {events_per_km if events_per_km is not None else 'Insufficient Data'}")
print(f"Hard Brake Ratio         : {hard_brake_ratio:.2f}")
print(f"Speed Variability        : {speed_variability:.2f}")
print("-" * 45)
print(f"Aggressiveness Score     : {aggressiveness_score} / 100")
print(f"Driver Grade             : {driver_grade}")


CWD: C:\Users\VIEW LIFE\driving-log-analysis\notebook CSV: \data\driving_log.csv EXISTS: False


FileNotFoundError: driving_log.csv not found. Place it under ./data/ (repo root 기준) and run this notebook from the repository root.